# User Defined Materials

## Overview

Matmodlab provides two methods of implementing user defined materials:

- subclassing the `matmodlab.mmd.MaterialModel` class, and
- interfacing with the Matmodlab Fortran `umat` API.

The former are referred to as "standard" materials and the latter "user" materials.  The builtin materials provided in the Matmodlab material library are definded using both methods.

For each implementation, the user material is called at each frame of every step. It is provided with the material state at the start of the increment (stress, solution-dependent state variables, temperature, etc) and with the increments in temperature, deformation, and time.

The implementation of both model types will be demonstrated with a standard isotropic linear elastic model.  

<a name='contents'></a>
## Contents

1. <a href='#linelast'>Isotropic Linear Elasticity</a>
2. <a href='#umat.std'>Standard Model Implementation</a>
3. <a href='#umat.user'>Fortran API Implementation</a>
4. <a href='#umat.compare'>Model Comparison</a>
5. <a href='#conc'>Conclusion</a>

<a name='linelast'></a>
## Isotropic Linear Elasticity

The mechanical response of a linear elastic material is defined by

$$
\pmb{\sigma} = \mathbb{C}{:}\pmb{\epsilon} = 3K\pmb{\epsilon}^{\rm iso} + 2G\pmb{\epsilon}^{\rm dev}
$$

where $K$ is the bulk modulus and $G$ is the shear modulus.  The strain $\pmb{\epsilon}$ can be determined from the deformation gradient $\pmb{F}$ as

$$
\pmb{\epsilon} = \frac{1}{2\kappa}\left[\left(\pmb{F}^{\rm T}{\cdot}\pmb{F}\right)^{2\kappa} - \pmb{I}\right]
$$

where $\kappa$ is the generalized [Seth-Hill](https://en.wikipedia.org/wiki/Finite_strain_theory#Seth-Hill_family_of_generalized_strain_tensors) strain parameter.  Defined as such, several well known finite strain measures are emitted:

- $\kappa=1$: Green-Lagrange reference strain
- $\kappa=-1$: Alamansi spatial strain
- $\kappa=0$: Logarithmic, or true, strain

The implementations of linear elasticity to follow will take as input Young's modulus `E`, Poisson's ratio `Nu`, and the Seth-Hill parameter `k` for changing the strain definition.

<a name='umat.std'></a>
## Standard Material Implementation

Material models implemented as subclasses of the `MaterialModel` class are referred to as "standard" materials.  Minimally, the materials installed as standard materials must define

- `name`: *class attribute*

   Used for referencing the material model in the `MaterialPointSimulator`.
   
- `param_names`: *class method*

   Used by the the `MaterialPointSimulator` for parsing input parameter names and assembling a material parameters array.
   
- `update_state`: *instance method*

   Updates the material stress, stiffness (optional), and state dependent variables to the end of the time increment.  If the stiffness is returned as `None`, Matmodlab will determine it numerically.
   
Other optional methods include

   
- `setup`: *instance method [optional]*

   Checks goodness of user input and requests storage allocation for state dependent variables.
   
In the example below, in addition to some standard functions imported from `Numpy`, several helper functions are imported from various locations in Matmodlab:

- `matmodlab.utils.mmlabpack`

   - `logm`, `powm`: computes the matrix logarithm and power
   - `asarray`: converts a symmetric tensor stored as a 3x3 matrix to an array of length 6
   - `polar_decomp`: computes the polar decomposition of the deformation gradient $\pmb{F}$
   - `iso`, `dev`: computes the isotropic and deviatoric parts of a second-order symmetric tensor stored as an array of length 6
- `matmodlab.constants`
   - `VOIGT`: mulitplier for converting tensor strain components to engineering strain components
   
The relevant input parameters to the `update_state` method from Matmodlab are:

- `F`: the deformation gradient at the end of the step
   
### Loading the Material Model

Once defined in a computational cell, the material model is loaded in to Matmodlab through the `load_material` function using the `std_material` keyword.

In [1]:
%matmodlab

BokehJS successfully loaded.

Populating the interactive namespace from matmodlab and bokeh


In [2]:
import logging
from numpy import dot, zeros, ix_, eye
from matmodlab.mmd.material import MaterialModel
from matmodlab.utils.mmlabpack import logm, powm, asarray, polar_decomp, iso, dev
from matmodlab.constants import VOIGT

class UserElastic(MaterialModel):
    name = "uelastic-std"

    @classmethod
    def param_names(cls, n):
        return ['k', 'E', 'Nu']

    def setup(self, **kwargs):
        """Set up the Elastic material

        """
        logger = logging.getLogger('matmodlab.mmd.simulator')

        # Check inputs
        E = self.parameters['E']
        Nu = self.parameters['Nu']

        errors = 0
        if E <= 0.0:
            errors += 1
            logger.error("Young's modulus E must be positive")
        if Nu > 0.5:
            errors += 1
            logger.error("Poisson's ratio > .5")
        if Nu < -1.0:
            errors += 1
            logger.error("Poisson's ratio < -1.")
        if Nu < 0.0:
            logger.warn("#---- WARNING: negative Poisson's ratio")
        if errors:
            raise ValueError("stopping due to previous errors")

    def update_state(self, time, dtime, temp, dtemp, energy, rho, F0, F,
        stran, d, elec_field, stress, statev, **kwargs):
        """Compute updated stress given strain increment"""

        # elastic properties
        k = self.parameters['k']
        E = self.parameters['E']
        Nu = self.parameters['Nu']

        # Get the bulk, shear, and Lame constants
        K = E / 3. / (1. - 2. * Nu)
        G = E / 2. / (1. + Nu)

        K3 = 3. * K
        G2 = 2. * G
        Lam = (K3 - G2) / 3.

        # elastic stiffness
        ddsdde = zeros((6,6))
        ddsdde[ix_(range(3), range(3))] = Lam
        ddsdde[range(3),range(3)] += G2
        ddsdde[range(3,6),range(3,6)] = G

        R, U = polar_decomp(F.reshape(3,3))
        if abs(k) <= 1e-12:
            e = logm(U)
        else:
            e = 1. / 2 / k * (powm(U, 2*k) - eye(3))

        # convert strain to an array
        e = asarray(e, 6)

        # stress update
        stress = K3 * iso(e) + G2 * dev(e)

        return stress, statev, ddsdde
load_material(std_material=UserElastic)

### Verification Test

Exercising the elastic model through a path of uniaxial stress should result in the slope of axial stress vs. axial strain being equal to the input parameter `E`.

**Note:** the input parameters to a standard material are given as a dictionary of `name:value` pairs for each paramter.  Parameters not specified are initialized to a value of zero.

In [3]:
mps1 = MaterialPointSimulator('uelastic-std')
mps1.Material('uelastic-std', {'E': 10e6, 'Nu': .333})
mps1.MixedStep(components=(.1, 0, 0), descriptors='ESS', frames=50)
i = where(mps1.E.XX > 0.)
E = mps1.S.XX[i] / mps1.E.XX[i]
assert allclose(E, 10e6, atol=1e-3, rtol=1e-3)

<a name='umat.user'></a>
## Fortran API Implementation

Material models implemented through the Fortran API are referred to as "user" materials.  The user material must provide a `umat` subroutine with the following signature:

```fortran
subroutine umat(stress, statev, ddsdde, sse, spd, scd, rpl, &
     ddsddt, drplde, drpldt, stran, dstran, time, dtime, temp, dtemp, &
     predef, dpred, cmname, ndi, nshr, ntens, nstatv, props, nprops, &
     coords, drot, pnewdt, celent, dfgrd0, dfgrd1, noel, npt, layer, &
     kspt, kstep, kinc)
```

Isotropic linear elasticity has been implemented in `umat_linear_elastic.f90`.  The parameters relevant to the material material are:

- `props` (input): material parameter array
- `dfgrd1` (input): deformation gradient
- `ddsdde` (output): material tangent stiffness
- `stress` (output): material stress

Utility procedures, linked to all user materials, used in the linear elastic model are

- `stdb_abqerr(iop, msg, intv, realv, charv)`: writes messages to the output file
- `sprind(S, AL, AV, imul, ndir, nshr)`: computes eigenvalues and eigenvectors

Full descriptions of these, and other utility procedures are given in the full documentation.

### Loading the Material Model

Like the standard material, the user material is loaded in to Matmodlab by the `load_material` function but using the `user_material` keyword to point to the Fortran file containing the material model definitions (coding).  Additionally, the `name` keyword is required for user materials so that the material can be referenced by name throughout Matmodlab.

In [4]:
filename = 'umat_linear_elastic.f90'
print open(filename).read()
load_material(user_material=filename, name='uelastic_user')

! --------------------------------------------------------------------------- !
!    UMAT FOR ISOTROPIC ELASTICITY
!
!    PROPS(1) - KAPPA
!    PROPS(2) - E
!    PROPS(3) - NU
! --------------------------------------------------------------------------- !
subroutine umat(stress, statev, ddsdde, sse, spd, scd, rpl, &
     ddsddt, drplde, drpldt, stran, dstran, time, dtime, temp, dtemp, &
     predef, dpred, cmname, ndi, nshr, ntens, nstatv, props, nprops, &
     coords, drot, pnewdt, celent, dfgrd0, dfgrd1, noel, npt, layer, &
     kspt, kstep, kinc)
  implicit none
  character*8, intent(in) :: cmname
  integer, intent(in) :: ndi, nshr, ntens, nstatv, nprops
  integer, intent(in) :: noel, npt, layer, kspt, kstep, kinc
  real(8), intent(in) :: sse, spd, scd, rpl, drpldt, time, dtime, temp, dtemp
  real(8), intent(in) :: pnewdt, celent
  real(8), intent(inout) :: stress(ntens), statev(nstatv), ddsdde(ntens, ntens)
  real(8), intent(inout) :: ddsddt(ntens), drplde(ntens)
  real(8), intent(

### Verification Test

Exercising the elastic model through a path of uniaxial stress should result in the slope of axial stress vs. axial strain being equal to the input parameter `E`.

**Note:** the input parameters to a user material are given as a sequence of values given in the order expected by the material.

In [5]:
environ.verbosity = 5
mps2 = MaterialPointSimulator('uelastic_user')
mps2.Material('uelastic_user', [0, 10e6, .333])
mps2.MixedStep(components=(.1, 0, 0), descriptors='ESS', frames=50)
i = where(mps2.E.XX > 0.)
E = mps2.S.XX[i] / mps2.E.XX[i]
assert allclose(E, 10e6, atol=1e-3, rtol=1e-3)

<a name='umat.compare'></a>
## Material Model Comparison

The response of each material should be identical, to within the precision of each implementation.

In [6]:
p = create_figure()
p.line(mps1.E.XX, mps1.S.XX, line_width=8, color='orange')
p.line(mps2.E.XX, mps2.S.XX, line_width=2, color='blue')
show(p)

<a name='conc'></a>
## Conclusion

Several methods exist for defining user materials, two of which were outlined in this notebook.